In [128]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import linear_model, metrics, ensemble
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
import math

In [3]:
data = pd.read_csv("./chevron.csv")
# Clean data:
# get rid of zeros
# make sure that empty cells is filled in with 0
# drop non states (US, etc.)
# lots of 0's
# get rid of random features that are duplicates
# drop conversion factor columns

## Filter stuff out
# states
## dropping DC, US, Gulf of Mexico, 
## factors

## Convert stuff to billion btu
# fuel ethanol

# add additional data
## add CO2 emissions data
## some other stuff?
## aggregate data?

# normalize the features
# adjust the weights
# LASSO, some other method
## normalize the data
## replace zeros with means

# other thoughts:
## LP normalization: is this what we want?
## if everything is in the same unit... then do we need to normalize?
## 

In [4]:
print(data.head())
print(data.shape)

   Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0           0  BDFDB        AK  2015     21.0  Alaska            35.027804   
1           1  BDPRP        AK  2015      4.0  Alaska            35.027804   
2           2  BFFDB        AK  2015     21.0  Alaska            35.027804   
3           3  BFPRP        AK  2015      4.0  Alaska            35.027804   
4           4  CLPRB        AK  2015  17747.0  Alaska            35.027804   

   TotalNumberofInvestments  TotalAmountofAssistance  
0                      16.0                3345612.0  
1                      16.0                3345612.0  
2                      16.0                3345612.0  
3                      16.0                3345612.0  
4                      16.0                3345612.0  
(7595, 9)


In [5]:
# Clean the data
# Drop unnecessary rows

In [6]:
# Drop DC, US, Gulf of Mexico, Pacific
clean_df = data.query("StateCode != 'DC'").query("StateCode != 'US'").query("StateCode != 'X3'").query("StateCode != 'X5'")
print(clean_df.head())
print(clean_df.shape)
print(clean_df.StateCode.unique())
print(clean_df.StateCode.unique().size)

   Unnamed: 0    MSN StateCode  Year   Amount   State  CO2 Emissions (Mmt)  \
0           0  BDFDB        AK  2015     21.0  Alaska            35.027804   
1           1  BDPRP        AK  2015      4.0  Alaska            35.027804   
2           2  BFFDB        AK  2015     21.0  Alaska            35.027804   
3           3  BFPRP        AK  2015      4.0  Alaska            35.027804   
4           4  CLPRB        AK  2015  17747.0  Alaska            35.027804   

   TotalNumberofInvestments  TotalAmountofAssistance  
0                      16.0                3345612.0  
1                      16.0                3345612.0  
2                      16.0                3345612.0  
3                      16.0                3345612.0  
4                      16.0                3345612.0  
(7250, 9)
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA

In [7]:
# Convert everything to billion btu
## Convert biodiesel to billion btu
clean_df = clean_df
# Remove the BDFDB, BFFDB, BFPRP, EMFDB
clean_df = clean_df.query("MSN != 'BDFDB'").query("MSN != 'BFFDB'").query("MSN != 'BFPRP'").query("MSN != 'EMFDB'")
print(clean_df.shape)
# Convert biodiesel to energy
biodiesel_converted = clean_df.query("MSN == 'BDPRP'")
# print(biodiesel_converted['Amount'])
biodiesel_converted.loc[:, "Amount"] = biodiesel_converted.loc[:, "Amount"] * 5.46
print(biodiesel_converted)
non_bdprp = clean_df.query("MSN != 'BDPRP'")
clean_df = pd.concat([non_bdprp, biodiesel_converted])
print(clean_df.shape)


(6250, 9)
      Unnamed: 0    MSN StateCode  Year   Amount          State  \
1              1  BDPRP        AK  2015    21.84         Alaska   
30            30  BDPRP        AL  2015  1943.76        Alabama   
59            59  BDPRP        AR  2015  4449.90       Arkansas   
88            88  BDPRP        AZ  2015    10.92        Arizona   
117          117  BDPRP        CA  2015  4078.62     California   
...          ...    ...       ...   ...      ...            ...   
7440        7440  BDPRP        VT  2019     0.00        Vermont   
7469        7469  BDPRP        WA  2019  9871.68     Washington   
7498        7498  BDPRP        WI  2019  3510.78      Wisconsin   
7527        7527  BDPRP        WV  2019     0.00  West Virginia   
7556        7556  BDPRP        WY  2019     0.00        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
1               35.027804                      16.0                3345612.0  
30             119.200889  

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
# Convert ethanol to energy
ethanol_converted = clean_df.query("MSN == 'ENPRP'")
print(ethanol_converted['Amount'])
ethanol_converted.loc[:, "Amount"] = ethanol_converted.loc[:, "Amount"] * 3.192
print(ethanol_converted['Amount'])
non_enprp = clean_df.query("MSN != 'ENPRP'")
clean_df = pd.concat([non_enprp, ethanol_converted])
print(clean_df.shape)

9           0.0
38          0.0
67          0.0
96       1154.0
125      4650.0
         ...   
7448        0.0
7477        0.0
7506    15000.0
7535        0.0
7564        0.0
Name: Amount, Length: 250, dtype: float64
9           0.000
38          0.000
67          0.000
96       3683.568
125     14842.800
          ...    
7448        0.000
7477        0.000
7506    47880.000
7535        0.000
7564        0.000
Name: Amount, Length: 250, dtype: float64
(6250, 9)


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [9]:
clean_df = clean_df.query("MSN != 'CLPRK'").query("MSN != 'CLPRP'").query("MSN != 'COPRK'") \
                    .query("MSN != 'PAPRP'").query("MSN != 'NGMPK'").query("MSN != 'NGMPP'")
print(clean_df)


      Unnamed: 0    MSN StateCode  Year    Amount          State  \
4              4  CLPRB        AK  2015   17747.0         Alaska   
10            10  GETCB        AK  2015     186.0         Alaska   
11            11  HYTCB        AK  2015   14621.0         Alaska   
12            12  NCPRB        AK  2015   16298.0         Alaska   
13            13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
7448        7448  ENPRP        VT  2019       0.0        Vermont   
7477        7477  ENPRP        WA  2019       0.0     Washington   
7506        7506  ENPRP        WI  2019   47880.0      Wisconsin   
7535        7535  ENPRP        WV  2019       0.0  West Virginia   
7564        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
4               35.027804                      16.0                3345612.0  
10              35.027804

In [11]:
# Sort and reindex dataframe
print(clean_df.sort_values('Unnamed: 0'))
clean_df.reset_index(drop=True, inplace=True)

      Unnamed: 0    MSN StateCode  Year    Amount    State  \
1              1  BDPRP        AK  2015     21.84   Alaska   
4              4  CLPRB        AK  2015  17747.00   Alaska   
9              9  ENPRP        AK  2015      0.00   Alaska   
10            10  GETCB        AK  2015    186.00   Alaska   
11            11  HYTCB        AK  2015  14621.00   Alaska   
...          ...    ...       ...   ...       ...      ...   
7579        7579  WDPRB        WY  2019   4969.00  Wyoming   
7580        7580  WDTCB        WY  2019   4969.00  Wyoming   
7581        7581  WSTCB        WY  2019      0.00  Wyoming   
7582        7582  WWPRB        WY  2019   4969.00  Wyoming   
7583        7583  WYTCB        WY  2019  37070.00  Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
1               35.027804                      16.0                3345612.0  
4               35.027804                      16.0                3345612.0  
9               35

In [12]:
print(clean_df)

      Unnamed: 0    MSN StateCode  Year    Amount          State  \
0              4  CLPRB        AK  2015   17747.0         Alaska   
1             10  GETCB        AK  2015     186.0         Alaska   
2             11  HYTCB        AK  2015   14621.0         Alaska   
3             12  NCPRB        AK  2015   16298.0         Alaska   
4             13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
4745        7448  ENPRP        VT  2019       0.0        Vermont   
4746        7477  ENPRP        WA  2019       0.0     Washington   
4747        7506  ENPRP        WI  2019   47880.0      Wisconsin   
4748        7535  ENPRP        WV  2019       0.0  West Virginia   
4749        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804

In [69]:
# See how many zeroes there are in the dataframe
zeros_df = clean_df.query("Amount == 0")
print(zeros_df)

      Unnamed: 0    MSN StateCode  Year  Amount          State  \
16            16  NUETB        AK  2015     0.0         Alaska   
23            23  WDEXB        AK  2015     0.0         Alaska   
52            52  WDEXB        AL  2015     0.0        Alabama   
57            57  WYTCB        AL  2015     0.0        Alabama   
81            81  WDEXB        AR  2015     0.0       Arkansas   
...          ...    ...       ...   ...     ...            ...   
7390        7390  ENPRP        UT  2019     0.0           Utah   
7448        7448  ENPRP        VT  2019     0.0        Vermont   
7477        7477  ENPRP        WA  2019     0.0     Washington   
7535        7535  ENPRP        WV  2019     0.0  West Virginia   
7564        7564  ENPRP        WY  2019     0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
16              35.027804                      16.0                3345612.0  
23              35.027804                      16

In [29]:
# Test Query Code
test_query = clean_df.query("StateCode == 'AK' & Year == 2015")
print(test_query)
print(test_query.iloc[0, 8])
print(type(test_query.iloc[0, 8]))

# See how many features we have remaining
print(clean_df.MSN.unique())
print(clean_df.MSN.unique().size)

      Unnamed: 0    MSN StateCode  Year      Amount   State  \
0              4  CLPRB        AK  2015    17747.00  Alaska   
1             10  GETCB        AK  2015      186.00  Alaska   
2             11  HYTCB        AK  2015    14621.00  Alaska   
3             12  NCPRB        AK  2015    16298.00  Alaska   
4             13  NGMPB        AK  2015   380887.00  Alaska   
5             16  NUETB        AK  2015        0.00  Alaska   
6             17  PAPRB        AK  2015  1007501.00  Alaska   
7             19  REPRB        AK  2015    23807.00  Alaska   
8             20  SOTCB        AK  2015        3.00  Alaska   
9             21  TEPRB        AK  2015  1429942.00  Alaska   
10            22  TETCB        AK  2015   614242.00  Alaska   
11            23  WDEXB        AK  2015        0.00  Alaska   
12            24  WDPRB        AK  2015     6865.00  Alaska   
13            25  WDTCB        AK  2015     6865.00  Alaska   
14            26  WSTCB        AK  2015      622.00  Al

In [33]:
# alaska_df = clean_df.loc[clean_df["StateCode"] == "AK"]
print(clean_df)
print(clean_df.StateCode.unique())
print(clean_df.StateCode.unique().size)

      Unnamed: 0    MSN StateCode  Year    Amount          State  \
0              4  CLPRB        AK  2015   17747.0         Alaska   
1             10  GETCB        AK  2015     186.0         Alaska   
2             11  HYTCB        AK  2015   14621.0         Alaska   
3             12  NCPRB        AK  2015   16298.0         Alaska   
4             13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
4745        7448  ENPRP        VT  2019       0.0        Vermont   
4746        7477  ENPRP        WA  2019       0.0     Washington   
4747        7506  ENPRP        WI  2019   47880.0      Wisconsin   
4748        7535  ENPRP        WV  2019       0.0  West Virginia   
4749        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804

In [34]:
### Test on all states
# Generate all years and all states
all_years = [2015, 2016, 2017, 2018, 2019]
all_states = clean_df.StateCode.unique()

In [39]:
print(clean_df)
print(all_states)

      Unnamed: 0    MSN StateCode  Year    Amount          State  \
0              4  CLPRB        AK  2015   17747.0         Alaska   
1             10  GETCB        AK  2015     186.0         Alaska   
2             11  HYTCB        AK  2015   14621.0         Alaska   
3             12  NCPRB        AK  2015   16298.0         Alaska   
4             13  NGMPB        AK  2015  380887.0         Alaska   
...          ...    ...       ...   ...       ...            ...   
4745        7448  ENPRP        VT  2019       0.0        Vermont   
4746        7477  ENPRP        WA  2019       0.0     Washington   
4747        7506  ENPRP        WI  2019   47880.0      Wisconsin   
4748        7535  ENPRP        WV  2019       0.0  West Virginia   
4749        7564  ENPRP        WY  2019       0.0        Wyoming   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804

In [199]:
# Create Response Variable

num_datapoints = len(all_years) * len(all_states) 
num_cols = 1
y = pd.DataFrame(index=range(num_datapoints), columns=["TotalAmountofAssistance"])
investment_vals = []
for year in all_years:
    for state in all_states:
        state_query = "StateCode == " + "'" + str(state) + "'"
        year_query = "Year == " + str(year)
        investment_year_state = clean_df.query(year_query + " & " + state_query)
        # print("INVESTMENT YEAR QUERY DF")
        # print(investment_year.head())
        # print(investment_year.shape)
        investment_year_val = investment_year_state.iloc[0, 8]
        investment_vals.append(investment_year_val)
for yIdx in range(num_datapoints):
    y.at[yIdx, "TotalAmountofAssistance"] = investment_vals[yIdx]
# y.at[0]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][0]
# print(y)
# y.at[1]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][1519]
# y.at[2]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][3038]
# y.at[3]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][4557]
# y.at[4]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][6076]
# for investment_idx in range(5):
#     print(investment_idx)
#     print(data["TotalAmountofAssistance"][3038])
#     row_idx = 1519 * investment_idx
#     y.at[investment_idx]["TotalAmountofAssistance"] = data["TotalAmountofAssistance"][row_idx]
print(y)

# y.to_csv('assistance.csv')

    TotalAmountofAssistance
0                 3345612.0
1                 1864412.0
2                13453887.0
3                 1724853.0
4                30077512.0
..                      ...
245               1121855.0
246              18389113.0
247               2915140.0
248                790383.0
249                570570.0

[250 rows x 1 columns]


In [64]:
# num_datapoints = len(all_years) * len(all_states) 
# num_features = 19

# all_msns = clean_df.MSN.unique()

# X = pd.DataFrame(index=range(num_datapoints), columns=all_msns)

# for yearIdx in range(len(all_years)):
#     for stateIdx in range(len(all_states)):
#         dataPtIdx = stateIdx * (yearIdx + 1)
#         # print(dataPtIdx)
#         curYear = all_years[yearIdx]
#         curState = all_states[stateIdx]
#         for msnIdx in range(len(all_msns)):
#             state_query = "StateCode == " + "'" + str(curState) + "'"
#             year_query = "Year == " + str(curYear)
#             msn_query = "MSN == " + "'" + str(all_msns[msnIdx]) + "'"
#             state_year_msn = clean_df.query(year_query + " & " + state_query + " & " + msn_query)
#             # print(state_year_msn)
#             amount_val = state_year_msn.iloc[0, 4]
#             X.at[dataPtIdx, all_msns[msnIdx]] = amount_val

# print(X)

         CLPRB  GETCB     HYTCB     NCPRB      NGMPB     NUETB     PAPRB  \
0      14867.0  186.0   14456.0   15951.0   364363.0       0.0  968358.0   
1     331420.0  141.0   91902.0   92144.0   182481.0  438728.0   55421.0   
2     247632.0  141.0   64484.0   65056.0   176290.0  417334.0   46417.0   
3     326748.0  141.0   85103.0   87057.0   159686.0  446097.0   39100.0   
4     370533.0  141.0  101447.0  105013.0   149607.0  412584.0   33591.0   
..         ...    ...       ...       ...        ...       ...       ...   
245  4828513.0  663.0    8838.0   48259.0  1684374.0       0.0  582170.0   
246        NaN    NaN       NaN       NaN        NaN       NaN       NaN   
247        NaN    NaN       NaN       NaN        NaN       NaN       NaN   
248        NaN    NaN       NaN       NaN        NaN       NaN       NaN   
249        NaN    NaN       NaN       NaN        NaN       NaN       NaN   

        REPRB   SOTCB      TEPRB      TETCB   WDEXB     WDPRB     WDTCB  \
0     22754.

In [60]:
# Test querying features

# all_msns = clean_df.MSN.unique()
# print(all_msns)
# X = pd.DataFrame(index=range(250), columns=all_msns)
# data_pt_idx= 0
# clean_df_row_idx = 0
# for year in all_years:
#     for state in all_states:
#         # data point
#         for feature_idx in range(19):
#             X.at[data_pt_idx, all_msns[feature_idx]] = clean_df.at[clean_df_row_idx, "Amount"]
#             clean_df_row_idx += 1
#         data_pt_idx += 1
# print(X)

['CLPRB' 'GETCB' 'HYTCB' 'NCPRB' 'NGMPB' 'NUETB' 'PAPRB' 'REPRB' 'SOTCB'
 'TEPRB' 'TETCB' 'WDEXB' 'WDPRB' 'WDTCB' 'WSTCB' 'WWPRB' 'WYTCB' 'BDPRP'
 'ENPRP']
         CLPRB      GETCB      HYTCB      NCPRB       NGMPB      NUETB  \
0      17747.0      186.0    14621.0    16298.0    380887.0        0.0   
1      91902.0    92144.0   182481.0   438728.0     55421.0   258955.0   
2    1031466.0   144716.0    35611.0   117840.0       124.0  1331452.0   
3        212.0   128954.0    48229.0   615867.0   1495858.0        0.0   
4     211839.0  2352883.0  7525491.0        0.0     98852.0    98852.0   
..         ...        ...        ...        ...         ...        ...   
245  11034.744        0.0        0.0        0.0    7702.296        0.0   
246   3801.672        0.0        0.0  39076.464  158715.816        0.0   
247        0.0   3466.512        0.0        0.0   48240.696        0.0   
248   5014.632  112549.92  80237.304  40940.592    2694.048        0.0   
249        0.0        0.0  112

In [87]:
# Create feature input matrix

X = pd.read_csv("flipped.csv")
# print(X)

sorted_X = X.sort_values(["Year", "State"], ascending=[True, True])
# print(sorted_X)

## edit sorted_X

sorted_X.loc[:, 'ENPRP'] *= 3.192
sorted_X.loc[:, 'BDPRP'] *= 5.46

## Remove state and year columns

# print(sorted_X)

sorted_X.drop(columns=['State', 'Year'], inplace=True)

print(sorted_X.head())

# print(sorted_X)
# print(sorted_X.columns.size)

      BDPRP     CLPRB      ENPRP     GETCB     HYTCB     NCPRB      NGMPB  \
0     21.84   17747.0      0.000     186.0   14621.0   16298.0   380887.0   
5   1943.76  331420.0      0.000     141.0   91902.0   92144.0   182481.0   
10  4449.90    1819.0      0.000     808.0   33261.0   34194.0  1031466.0   
15    10.92  146450.0   3683.568     345.0   60906.0  113693.0       99.0   
20  4078.62       0.0  14842.800  112878.0  128681.0  567365.0   271086.0   

       NUETB      PAPRB     REPRB     SOTCB      TEPRB      TETCB  WDEXB  \
0        0.0  1007501.0   23807.0       3.0  1429942.0   614242.0    0.0   
5   438728.0    55421.0  258955.0     101.0  1267004.0  1917138.0    0.0   
10  144716.0    35611.0  117840.0     124.0  1331452.0  1062359.0    0.0   
15  340153.0      212.0  128954.0   48229.0   615867.0  1495858.0    0.0   
20  193529.0  1150741.0  737527.0  211839.0  2352883.0  7525491.0    0.0   

       WDPRB     WDTCB    WSTCB     WWPRB     WYTCB  
0     6865.0    6865.0    

In [25]:
print(alaska_df)

      Unnamed: 0    MSN StateCode  Year    Amount   State  \
0              4  CLPRB        AK  2015   17747.0  Alaska   
1             10  GETCB        AK  2015     186.0  Alaska   
2             11  HYTCB        AK  2015   14621.0  Alaska   
3             12  NCPRB        AK  2015   16298.0  Alaska   
4             13  NGMPB        AK  2015  380887.0  Alaska   
...          ...    ...       ...   ...       ...     ...   
4500           9  ENPRP        AK  2015       0.0  Alaska   
4550        1528  ENPRP        AK  2016       0.0  Alaska   
4600        3047  ENPRP        AK  2017       0.0  Alaska   
4650        4566  ENPRP        AK  2018       0.0  Alaska   
4700        6085  ENPRP        AK  2019       0.0  Alaska   

      CO2 Emissions (Mmt)  TotalNumberofInvestments  TotalAmountofAssistance  
0               35.027804                      16.0                3345612.0  
1               35.027804                      16.0                3345612.0  
2               35.027804     

In [26]:
numRows = 5
numCols = 29

x_comb = pd.DataFrame(index=range(numRows), columns=range(numCols))
x_comb.columns = alaska_df["MSN"][0:numCols]

# x = pd.DataFrame(index=range(numRows), columns=range(numCols))
# x.columns = alaska_df["MSN"][0:numCols]
# x_test = pd.DataFrame(index=range(4, 5), columns=range(numCols))
# x_test.columns = alaska_df["MSN"][0:numCols]
# print(x.columns)
avgs = [[0] * numCols] * numRows
for row_idx in range(int(alaska_df.shape[0])):
    yr_idx = int(row_idx / numCols)
    x_comb.at[yr_idx, x_comb.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]
    # if row_idx < int(alaska_df.shape[0] * 4 / 5):
    #     x.at[yr_idx, x.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]
    # else:
    #     x_test.at[yr_idx, x.columns[row_idx % numCols]] = data.loc[1519 * yr_idx + row_idx]["Amount"]

norm_x_comb = (x_comb - x_comb.mean())/x_comb.std()

norm_x = norm_x_comb.head(4)
norm_x_test = norm_x_comb.tail(1)

# print(x)
# print(x_test)
print(norm_x)
print(norm_x_test)


MSN     BDFDB     BDPRP     BFFDB     BFPRP     CLPRB     CLPRK     CLPRP  \
0   -0.898188 -0.898173 -0.733353 -0.738738 -0.579055 -0.128243 -0.526746   
1   -0.296652 -0.296501 -0.586968 -0.586262   1.51242   0.94301  1.407429   
2    1.252345  1.251893 -0.210018 -0.193868 -0.732947  0.316033 -0.785824   
3    -0.90489 -0.905108 -0.210173 -0.220233  0.540098   0.51974   0.70079   

MSN     COPRK     EMFDB     ENPRP  ...     REPRB     SOTCB     TEPRB  \
0    0.350264 -0.580166 -0.579933  ... -0.677567 -0.542169  0.167092   
1    0.811137 -0.580166 -0.579933  ... -0.224796 -0.539738 -0.289706   
2    0.903312 -0.580166 -0.579933  ... -0.474328 -0.539675 -0.479445   
3   -0.663658  0.011308   0.01037  ... -0.388164 -0.140189 -1.017501   

MSN     TETCB     WDEXB     WDPRB     WDTCB     WSTCB     WWPRB     WYTCB  
0   -0.667942 -0.447214 -0.979919 -0.964505 -0.510625 -1.012773 -0.448595  
1   -0.224551 -0.447214  1.401078  1.419738 -0.454722  1.158346 -0.476136  
2   -0.517672  1.788854  

In [91]:
### Get testing data
y_test = pd.read_csv("assistance_test.csv")
x_test = pd.read_csv("2020flipped.csv")

sorted_X_test = x_test.sort_values(["Year", "State"], ascending=[True, True])
# print(sorted_X)

## edit sorted_X

sorted_X_test.loc[:, 'ENPRP'] *= 3.192
sorted_X_test.loc[:, 'BDPRP'] *= 5.46

## Remove state and year columns

# print(sorted_X)

sorted_X_test.drop(columns=['State', 'Year'], inplace=True)

print(sorted_X_test.head())
print(y_test.head())

     BDPRP     CLPRB    ENPRP     GETCB     HYTCB     NCPRB     NGMPB  \
0     0.00   15523.0     0.00     186.0   15475.0   16851.0  376283.0   
1  1567.02  313282.0     0.00     141.0  117112.0  120692.0  124831.0   
2  8479.38       0.0     0.00     808.0   39747.0   43767.0  489956.0   
3     0.00       0.0     0.00     345.0   56354.0  142484.0      68.0   
4  4892.16       0.0  8650.32  101857.0  187545.0  845282.0  192081.0   

      NUETB     PAPRB      REPRB     SOTCB      TEPRB      TETCB   WDEXB  \
0       0.0  932482.0    24682.0      62.0  1348969.0   640132.0     0.0   
1  454756.0   24528.0   288515.0    3439.0  1205912.0  1838656.0  5592.0   
2  157288.0   23578.0   111538.0    3212.0   782360.0  1034661.0     0.0   
3  329471.0      28.0   152046.0   80134.0   481613.0  1508154.0     0.0   
4  169773.0  814462.0  1013850.0  436716.0  2190166.0  6922764.0     0.0   

      WDPRB     WDTCB    WSTCB     WWPRB     WYTCB  
0    7423.0    7423.0    407.0    7831.0    1128.0 

In [92]:
y_test.drop(columns=y_test.columns[0], axis=1, inplace=True)
print(y_test.head())

   TotalAmountofAssistance
0                9567164.0
1                1174323.0
2               61208016.0
3               23756565.0
4               58722209.0


In [88]:
reg = linear_model.LinearRegression()
reg.fit(sorted_X, y)

LinearRegression()

In [116]:
lasso_reg = Lasso(1)

In [121]:
lasso_reg.fit(sorted_X, y)

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.035e+17, tolerance: 5.035e+13
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.01)

In [123]:
lasso_train_score = lasso_reg.score(sorted_X, y)
print(lasso_train_score)

0.19139389064027312


In [125]:
y_pred_lasso = lasso_reg.predict(sorted_X_test)
print(y_pred_lasso)

[ 6.28008128e+06  2.47595029e+07  7.23373507e+06  3.18883211e+07
  8.47406309e+07  1.83937423e+07  5.59908506e+06  1.12340885e+07
  4.79993282e+07  8.55564880e+07  1.06501197e+07 -2.93103112e+05
  1.17165918e+07  1.61651065e+07  1.05468586e+07  1.31821433e+07
  1.52345455e+07  5.38795814e+07  1.02470337e+07  1.39255211e+07
  6.58171522e+05  2.23895053e+06  8.43113285e+06  1.29195691e+07
  4.75280385e+07  1.16675572e+07  1.26677558e+08 -7.20760757e+06
  1.11012465e+07  5.39321882e+06  1.75425739e+07  2.47327431e+06
  2.34118902e+07  1.34614269e+07  2.15969647e+07  1.36226926e+07
  1.38101632e+07  7.31705611e+06  9.02692406e+06  4.28068148e+07
  1.13647999e+07  2.32906077e+07  7.08482367e+07  1.76983260e+07
  4.17664542e+07  8.01834358e+06  2.18263733e+07  3.85506934e+06
  1.64453247e+07  5.13642480e+06]


In [126]:
print(lasso_reg.coef_)
print(lasso_reg.intercept_)

[-1.66133236e+02 -3.04511176e+00 -7.72338259e+01 -6.57342935e+01
 -2.64748518e+01  2.70051834e+01  6.43309891e-01 -8.95794657e+00
 -9.86714816e+00  2.59475621e+01  1.06246194e+02  1.20170588e+00
  7.92001511e+00  4.40448117e+03 -2.29125512e+02 -1.27158719e+01
 -8.90743259e+02  7.91958270e+01 -4.44933930e+01]
[9502865.62703156]


In [61]:
print(reg.intercept_)
print(reg.coef_)

[9304626.40874828]
[[  446070.88214614   445485.53466719   707433.75430459   706586.72209353
   -517087.19220269    51579.55147729  -310055.3419393  -1293046.64744555
    674952.8973394    673617.50995882    11148.79212505   149153.12733561
    258908.56090911  -453317.75058139 -1943887.78525199  -341171.8581966
    518574.50640231 -1552057.15946725 -1549762.78107775    46353.68936528
    457809.92894808 -1260433.68622962    59272.21460338   987989.40591331
  -1445936.0672543  -1484910.13242291    39043.89360253 -1300650.42063566
     81084.47730676]]


In [230]:
y_pred = reg.predict(sorted_X_test)
print(y_pred)
print(y_test)

[[-4.33020400e+06]
 [-1.03330080e+07]
 [ 7.11658245e+06]
 [ 3.00297071e+07]
 [ 8.59435946e+07]
 [ 8.95327864e+06]
 [ 5.04678280e+06]
 [ 1.07212881e+07]
 [ 4.81565702e+07]
 [ 8.41399954e+07]
 [ 1.82572662e+07]
 [ 2.41338449e+06]
 [ 1.11145715e+07]
 [ 1.62739063e+07]
 [ 1.12047407e+07]
 [ 2.22663062e+07]
 [ 1.62693483e+07]
 [ 5.08291914e+07]
 [ 1.78629382e+07]
 [ 1.30685582e+07]
 [ 8.67152049e+05]
 [ 5.13639080e+06]
 [ 9.58736445e+06]
 [ 1.32103001e+07]
 [ 1.13221420e+07]
 [ 5.98480166e+05]
 [ 1.22641695e+08]
 [-5.48347900e+06]
 [ 1.04617190e+07]
 [-1.93029445e+06]
 [ 1.49353918e+07]
 [ 1.60706807e+06]
 [ 2.58293400e+07]
 [ 1.22444801e+07]
 [ 1.98138317e+07]
 [ 1.48457573e+07]
 [ 1.26961874e+07]
 [-2.55032455e+06]
 [ 8.23504794e+06]
 [ 7.05127792e+06]
 [ 1.14040192e+07]
 [ 2.24490801e+07]
 [ 6.00356406e+07]
 [ 6.77896391e+06]
 [ 7.57952052e+07]
 [ 7.37442691e+06]
 [ 1.90488309e+07]
 [ 4.55344024e+06]
 [ 1.45327220e+07]
 [ 3.42095905e+06]]
    TotalAmountofAssistance
0                 956

In [129]:
print(reg.intercept_)
print(reg.coef_)
print(reg.score(sorted_X, y))
print(sorted_X.columns)

[8848932.95507227]
[[ 5.64273389e+02 -1.57101119e+06  1.20344927e+03 -1.41370796e+05
  -1.41406391e+05  1.42115466e+05 -1.57100750e+06 -1.57101612e+06
  -1.57101750e+06 -1.57169610e+06 -1.41281992e+05  1.57100905e+06
   7.82328927e+00 -2.65199001e+07  3.49553490e+07 -2.65241724e+07
   8.43052956e+06 -8.43061306e+06 -1.41415516e+05]]
0.21969392167899515
Index(['BDPRP', 'CLPRB', 'ENPRP', 'GETCB', 'HYTCB', 'NCPRB', 'NGMPB', 'NUETB',
       'PAPRB', 'REPRB', 'SOTCB', 'TEPRB', 'TETCB', 'WDEXB', 'WDPRB', 'WDTCB',
       'WSTCB', 'WWPRB', 'WYTCB'],
      dtype='object')


In [98]:
mse = metrics.mean_squared_error(y_pred, y_test)
print(mse)
print(math.sqrt(mse))

1054264821397483.4
32469444.426991407


In [127]:
mse_lasso = metrics.mean_squared_error(y_pred_lasso, y_test)
print(mse_lasso)
print(math.sqrt(mse_lasso))

873354863201628.4
29552577.94510706


In [102]:
### Gradient Boosting
params = {"n_estimators": 3, "max_depth": 3, "learning_rate": 1, "criterion": "squared_error"}
gradient_boosting_regressor_model = ensemble.GradientBoostingRegressor(**params)
gradient_boosting_regressor_model.fit(sorted_X, y)

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(criterion='squared_error', learning_rate=1,
                          n_estimators=3)

In [103]:
print(gradient_boosting_regressor_model.score(sorted_X, y))

0.8594088494586949


In [109]:
y_pred_gradient = gradient_boosting_regressor_model.predict(sorted_X_test)
print(y_pred_gradient)
print(y_test)

[2.79923611e+06 1.08376685e+06 2.32641093e+07 4.25624095e+07
 4.56641589e+07 2.79923611e+06 2.79923611e+06 2.79923611e+06
 4.25624095e+07 2.20424347e+08 2.79923611e+06 2.79923611e+06
 2.79923611e+06 2.79923611e+06 2.79923611e+06 2.79923611e+06
 2.32641093e+07 2.32641093e+07 2.79923611e+06 2.79923611e+06
 2.79923611e+06 2.79923611e+06 2.79923611e+06 2.79923611e+06
 2.32641093e+07 2.79923611e+06 1.26884709e+08 2.79923611e+06
 2.79923611e+06 2.79923611e+06 2.42604689e+08 2.79923611e+06
 2.42771305e+07 2.79923611e+06 2.79923611e+06 2.79923611e+06
 1.18795524e+08 2.79923611e+06 2.79923611e+06 2.32641093e+07
 2.79923611e+06 2.79923611e+06 4.56641589e+07 2.79923611e+06
 2.32641093e+07 2.79923611e+06 5.90098551e+06 2.79923611e+06
 5.91571667e+05 5.91571667e+05]
    TotalAmountofAssistance
0                 9567164.0
1                 1174323.0
2                61208016.0
3                23756565.0
4                58722209.0
5                 6304206.0
6                  491729.0
7           

In [110]:
mse = metrics.mean_squared_error(y_pred_gradient, y_test)

print(mse)

print(math.sqrt(mse))


2881435954827484.5
53679008.51196382


In [130]:
### Drop wood energy production
sorted_X_drop = sorted_X.drop(columns=['WDPRB'])
sorted_X_test_drop = sorted_X_test.drop(columns=['WDPRB'])

### Lin Reg Model Again
reg_drop = linear_model.LinearRegression()
reg_drop.fit(sorted_X_drop, y)

LinearRegression()

In [135]:
print(reg_drop.coef_)
print(reg_drop.intercept_)
print(reg_drop.score(sorted_X_drop, y))

[[-6.14656790e+02 -3.08993677e+06 -9.27032125e+02 -4.56783020e+05
  -4.56778040e+05  4.56299264e+05 -3.08993290e+06 -3.08993842e+06
  -3.08994309e+06 -3.08943032e+06 -4.56643965e+05  3.08993448e+06
   8.10309352e+00  6.42519838e+06  6.42067588e+06  6.41995886e+06
  -6.42131680e+06 -4.56793442e+05]]
[9209276.52562891]
0.19742345894701063


In [176]:
y_pred_drop = reg_drop.predict(sorted_X_test_drop)
print(y_pred_drop)
print(y_test)

y_pred_df = pd.DataFrame(y_pred_drop, columns=['PredictedAssistance'])
# print(y_pred_df)
# print(all_states)
# y_pred_df = y_pred_df.reindex(all_states)
# print(y_pred_df)
y_pred_df.to_csv("LinearRegPredictions.csv")

[[-3.52277429e+06]
 [ 2.64205289e+07]
 [ 7.43907022e+06]
 [ 3.29068962e+07]
 [ 8.60485784e+07]
 [ 1.11927834e+07]
 [ 4.98488535e+06]
 [ 1.14027345e+07]
 [ 4.95510178e+07]
 [ 8.82860963e+07]
 [ 1.65185702e+07]
 [-2.57567493e+06]
 [ 1.12613894e+07]
 [ 1.70204690e+07]
 [ 9.31373790e+06]
 [ 1.99474129e+07]
 [ 1.79615242e+07]
 [ 5.08337492e+07]
 [ 1.54244252e+07]
 [ 1.34503614e+07]
 [ 9.89819922e+04]
 [ 5.88091541e+06]
 [ 8.62328981e+06]
 [ 1.36890377e+07]
 [ 4.85495085e+07]
 [ 1.47921416e+06]
 [ 1.30140542e+08]
 [-4.87784994e+06]
 [ 7.91342378e+06]
 [ 1.46280694e+06]
 [ 1.42061327e+07]
 [ 1.74740784e+06]
 [ 2.73033438e+07]
 [ 1.25621721e+07]
 [ 1.82066303e+07]
 [ 1.56375533e+07]
 [ 1.27179846e+07]
 [-3.95629001e+05]
 [ 8.11603097e+06]
 [ 4.54173081e+07]
 [ 1.05607295e+07]
 [ 2.40409983e+07]
 [ 6.26259368e+07]
 [ 7.86722069e+06]
 [ 4.25481778e+07]
 [ 7.19222069e+06]
 [ 2.03479351e+07]
 [ 3.61204776e+06]
 [ 1.51284126e+07]
 [ 3.81306932e+06]]
    TotalAmountofAssistance
0                 956

In [134]:
mse = metrics.mean_squared_error(y_pred_drop, y_test)
print(math.sqrt(mse))

29391026.10287109


In [136]:
### Polynomial Regression Model
poly = PolynomialFeatures(degree=2, include_bias=False)

In [144]:
poly_features = poly.fit_transform(sorted_X_drop)

In [141]:
poly_reg_model = linear_model.LinearRegression()

In [145]:
poly_reg_model.fit(poly_features, y)

LinearRegression()

In [146]:
print(poly_reg_model.score(poly_features, y))

0.9452227862433634


In [148]:
poly_test_features = poly.fit_transform(sorted_X_test_drop)
poly_reg_y_pred = poly_reg_model.predict(poly_test_features)

[[ 1.83723727e+07]
 [-1.23020523e+08]
 [-9.30730734e+06]
 [ 1.30845973e+08]
 [ 3.60144565e+09]
 [-8.13498673e+07]
 [-1.08668593e+07]
 [ 1.13245035e+07]
 [-1.57853931e+08]
 [ 2.93541653e+08]
 [ 2.14523167e+07]
 [-1.45799908e+09]
 [ 1.28020656e+05]
 [-2.86731499e+08]
 [-3.00227819e+08]
 [-2.08087275e+08]
 [-7.80269873e+07]
 [ 9.09770447e+07]
 [-1.15009134e+06]
 [-1.36783153e+07]
 [-2.11351534e+06]
 [ 4.90647247e+07]
 [-3.43718123e+08]
 [ 2.47083727e+07]
 [-1.16080939e+08]
 [ 1.19631567e+07]
 [-3.53863915e+08]
 [ 1.54153749e+08]
 [-7.29894193e+07]
 [-2.46307633e+07]
 [ 2.46785487e+07]
 [-1.11496427e+08]
 [ 7.65927637e+08]
 [-2.22379243e+08]
 [-1.33516523e+08]
 [-1.40905707e+08]
 [-1.36549611e+08]
 [ 9.14223893e+08]
 [ 1.08514682e+07]
 [ 9.92161487e+07]
 [-1.61359339e+08]
 [-5.03019344e+05]
 [-2.14518367e+10]
 [-5.44330033e+07]
 [-1.86564913e+07]
 [ 1.01907167e+07]
 [-2.92113643e+08]
 [-3.92194353e+07]
 [-8.61176113e+07]
 [-7.24861233e+07]]


In [150]:
print(poly_reg_y_pred)
print(y_test)

[[ 1.83723727e+07]
 [-1.23020523e+08]
 [-9.30730734e+06]
 [ 1.30845973e+08]
 [ 3.60144565e+09]
 [-8.13498673e+07]
 [-1.08668593e+07]
 [ 1.13245035e+07]
 [-1.57853931e+08]
 [ 2.93541653e+08]
 [ 2.14523167e+07]
 [-1.45799908e+09]
 [ 1.28020656e+05]
 [-2.86731499e+08]
 [-3.00227819e+08]
 [-2.08087275e+08]
 [-7.80269873e+07]
 [ 9.09770447e+07]
 [-1.15009134e+06]
 [-1.36783153e+07]
 [-2.11351534e+06]
 [ 4.90647247e+07]
 [-3.43718123e+08]
 [ 2.47083727e+07]
 [-1.16080939e+08]
 [ 1.19631567e+07]
 [-3.53863915e+08]
 [ 1.54153749e+08]
 [-7.29894193e+07]
 [-2.46307633e+07]
 [ 2.46785487e+07]
 [-1.11496427e+08]
 [ 7.65927637e+08]
 [-2.22379243e+08]
 [-1.33516523e+08]
 [-1.40905707e+08]
 [-1.36549611e+08]
 [ 9.14223893e+08]
 [ 1.08514682e+07]
 [ 9.92161487e+07]
 [-1.61359339e+08]
 [-5.03019344e+05]
 [-2.14518367e+10]
 [-5.44330033e+07]
 [-1.86564913e+07]
 [ 1.01907167e+07]
 [-2.92113643e+08]
 [-3.92194353e+07]
 [-8.61176113e+07]
 [-7.24861233e+07]]
    TotalAmountofAssistance
0                 956

In [151]:
poly_reg_rmse = np.sqrt(metrics.mean_squared_error(y_test, poly_reg_y_pred))
print(poly_reg_rmse)

3096183506.0380974


In [160]:
### Interaction only
interaction_poly = PolynomialFeatures(interaction_only=True, include_bias=False)
interaction_features = interaction_poly.fit_transform(sorted_X_drop)

In [161]:
interaction_reg = linear_model.LinearRegression()
interaction_reg.fit(interaction_features, y)

LinearRegression()

In [163]:
print(interaction_reg.score(interaction_features, y))

0.8508430340641762


In [164]:
interaction_test_features = interaction_poly.fit_transform(sorted_X_test_drop)
interaction_reg_y_pred = interaction_reg.predict(interaction_test_features)

In [165]:
print(interaction_reg_y_pred)
print(y_test)

[[-7.25356334e+06]
 [-1.25671915e+08]
 [ 5.93714127e+07]
 [ 2.18958101e+08]
 [-6.20413054e+09]
 [ 5.25770447e+07]
 [ 1.56742447e+07]
 [ 1.30399795e+07]
 [-1.16735793e+07]
 [ 4.50884885e+08]
 [-2.17109173e+07]
 [ 3.35974165e+08]
 [ 1.98396687e+07]
 [-3.78414571e+08]
 [-3.88950251e+08]
 [-1.30542315e+08]
 [-6.24839473e+07]
 [ 1.34158357e+08]
 [-1.37359793e+07]
 [-2.16887633e+07]
 [ 4.12212656e+05]
 [-2.93096753e+07]
 [-7.15114033e+07]
 [ 1.95241967e+07]
 [-3.46202993e+07]
 [ 1.69474066e+06]
 [-2.88836331e+08]
 [ 7.07141327e+07]
 [ 2.27331605e+08]
 [-4.03508593e+07]
 [ 3.16222607e+07]
 [-4.98623793e+07]
 [ 5.55090127e+07]
 [-3.48484913e+07]
 [-5.36437553e+07]
 [-1.40424427e+08]
 [ 2.40811727e+07]
 [-1.07269355e+08]
 [ 5.71488916e+06]
 [ 4.58533327e+07]
 [ 7.28615247e+07]
 [-9.94664113e+07]
 [-1.03212987e+10]
 [-2.01121134e+06]
 [ 5.31227666e+06]
 [ 1.08391967e+07]
 [-2.26574571e+08]
 [ 3.67259666e+06]
 [ 1.48849941e+08]
 [ 6.86743247e+07]]
    TotalAmountofAssistance
0                 956

In [166]:
interaction_reg_rmse = np.sqrt(metrics.mean_squared_error(y_test, interaction_reg_y_pred))
print(interaction_reg_rmse)

1718642332.8903716


In [184]:
lasso_reg_drop = Lasso()
lasso_reg_drop.fit(sorted_X_drop, y)

/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.035e+17, tolerance: 5.035e+13
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [188]:
print(lasso_reg_drop.coef_)
print(lasso_reg_drop.intercept_)
print(lasso_reg_drop.score(sorted_X_drop, y))

[-1.66133145e+02 -3.04510876e+00 -7.72338829e+01 -6.57344977e+01
 -2.64749263e+01  2.70051903e+01  6.43309158e-01 -8.95796949e+00
 -9.86714498e+00  2.59475907e+01  1.06246166e+02  1.20170501e+00
  7.92001434e+00  4.17670683e+03 -2.40486148e+02 -8.89387685e+02
  7.78407660e+01 -4.44934329e+01]
[9502862.53902255]
0.19139417248137436


In [190]:
y_pred_lasso = lasso_reg_drop.predict(sorted_X_test_drop)
print(y_pred_lasso)
print(y_test)

[ 6.28007899e+06  2.47592659e+07  7.23373922e+06  3.18883093e+07
  8.47406416e+07  1.83937377e+07  5.59908552e+06  1.12340857e+07
  4.79993156e+07  8.55564428e+07  1.06501202e+07 -2.93099333e+05
  1.17165918e+07  1.61650845e+07  1.05468672e+07  1.31821399e+07
  1.52345481e+07  5.38795640e+07  1.02470425e+07  1.39255179e+07
  6.58184564e+05  2.23896264e+06  8.43113882e+06  1.29195718e+07
  4.75277800e+07  1.16675530e+07  1.26677468e+08 -7.20760660e+06
  1.11012406e+07  5.39321826e+06  1.75425704e+07  2.47327435e+06
  2.34118814e+07  1.34614290e+07  2.15969651e+07  1.36226926e+07
  1.38101645e+07  7.31705185e+06  9.02692282e+06  4.28065538e+07
  1.13647939e+07  2.32905947e+07  7.08482097e+07  1.76983221e+07
  4.17666613e+07  8.01834447e+06  2.18263644e+07  3.85508301e+06
  1.64453215e+07  5.13642845e+06]
    TotalAmountofAssistance
0                 9567164.0
1                 1174323.0
2                61208016.0
3                23756565.0
4                58722209.0
5                 

In [193]:
mse_lasso_drop = metrics.mean_squared_error(y_test, y_pred_lasso)
print(math.sqrt(mse_lasso_drop))

29552587.77832985


In [194]:
### Normalize the data
sorted_X_drop_norm = (sorted_X_drop - sorted_X_drop.mean()) / (sorted_X_drop.std())
sorted_X_test_drop_norm = (sorted_X_test_drop - sorted_X_test_drop.mean()) / (sorted_X_test_drop.std())

     BDPRP     CLPRB    ENPRP     GETCB     HYTCB     NCPRB     NGMPB  \
0     0.00   15523.0     0.00     186.0   15475.0   16851.0  376283.0   
1  1567.02  313282.0     0.00     141.0  117112.0  120692.0  124831.0   
2  8479.38       0.0     0.00     808.0   39747.0   43767.0  489956.0   
3     0.00       0.0     0.00     345.0   56354.0  142484.0      68.0   
4  4892.16       0.0  8650.32  101857.0  187545.0  845282.0  192081.0   

      NUETB     PAPRB      REPRB     SOTCB      TEPRB      TETCB   WDEXB  \
0       0.0  932482.0    24682.0      62.0  1348969.0   640132.0     0.0   
1  454756.0   24528.0   288515.0    3439.0  1205912.0  1838656.0  5592.0   
2  157288.0   23578.0   111538.0    3212.0   782360.0  1034661.0     0.0   
3  329471.0      28.0   152046.0   80134.0   481613.0  1508154.0     0.0   
4  169773.0  814462.0  1013850.0  436716.0  2190166.0  6922764.0     0.0   

      WDTCB    WSTCB     WWPRB     WYTCB  
0    7423.0    407.0    7831.0    1128.0  
1  159187.0   1487

In [195]:
print(sorted_X_drop_norm)
print(sorted_X_test_drop_norm)

        BDPRP     CLPRB     ENPRP     GETCB     HYTCB     NCPRB     NGMPB  \
0   -0.533821 -0.340082 -0.438343 -0.256144 -0.316994 -0.552139 -0.180050   
5   -0.286166  0.024479 -0.438343 -0.259039  0.353660 -0.129830 -0.296753   
10   0.036771 -0.358594 -0.438343 -0.216122 -0.155234 -0.452494  0.202621   
15  -0.535228 -0.190499 -0.370271 -0.245913  0.084673 -0.009846 -0.404030   
20  -0.011071 -0.360708 -0.164050  6.994911  0.672832  2.516187 -0.244635   
..        ...       ...       ...       ...       ...       ...       ...   
229 -0.536635 -0.360708 -0.438343 -0.266246 -0.340555 -0.542311 -0.404088   
234  0.735413 -0.360708 -0.438343 -0.195017  4.657928  2.980924 -0.404088   
239 -0.084242 -0.360708  0.446472 -0.228540 -0.239785 -0.407149 -0.404088   
244 -0.536635  2.454478 -0.438343 -0.266053 -0.312039 -0.476336  1.124101   
249 -0.536635  5.251144 -0.438343 -0.225452 -0.367180 -0.374181  0.586663   

        NUETB     PAPRB     REPRB     SOTCB     TEPRB     TETCB     WDEXB  

In [201]:
linreg_norm = linear_model.LinearRegression()
linreg_norm.fit(sorted_X_drop_norm, y)

LinearRegression()

In [202]:
print(linreg_norm.coef_)
print(linreg_norm.intercept_)
print(linreg_norm.score(sorted_X_drop_norm, y))

[[-4.77001990e+06 -2.65862329e+12 -5.01644681e+07 -7.09907584e+09
  -5.26356076e+10  8.19507061e+10 -5.25318961e+12 -6.90304169e+11
  -3.79947443e+12 -7.01601861e+11 -2.16074802e+10  9.70324874e+12
   1.76166938e+07  2.52341125e+10  2.93722471e+11  8.41509453e+10
  -3.47969275e+11 -4.36717117e+10]]
[16605972.65616275]
0.19742345894631597


In [203]:
y_pred_norm = linreg_norm.predict(sorted_X_test_drop_norm)
print(y_pred_norm)
print(y_test)

[[ 4.52871808e+11]
 [-4.43074544e+11]
 [ 3.12812226e+11]
 [ 1.72159453e+11]
 [ 2.39864082e+11]
 [ 3.41144430e+11]
 [ 2.68296797e+11]
 [ 3.37535049e+11]
 [ 1.88415868e+11]
 [ 1.13533426e+11]
 [ 3.36736876e+11]
 [ 1.83125123e+11]
 [ 3.07278308e+11]
 [-1.34807140e+12]
 [-4.70979652e+11]
 [ 2.91225024e+11]
 [-6.55413996e+11]
 [ 6.37446151e+11]
 [ 3.37328199e+11]
 [ 2.19868741e+11]
 [ 3.17483086e+11]
 [ 1.91408083e+11]
 [ 2.21919520e+11]
 [ 2.75777911e+11]
 [ 2.51147186e+11]
 [-4.86346249e+11]
 [ 6.58407088e+10]
 [ 8.91443275e+10]
 [ 2.52582017e+11]
 [ 2.88462680e+11]
 [ 2.20752545e+11]
 [ 4.97286879e+11]
 [ 3.17769645e+11]
 [ 9.69521355e+10]
 [ 4.10332945e+11]
 [ 7.90399367e+11]
 [ 2.44441230e+11]
 [-6.80272935e+11]
 [ 3.37861434e+11]
 [ 5.58279229e+10]
 [ 2.88440887e+11]
 [ 1.26981354e+11]
 [ 2.04198445e+12]
 [-1.30293743e+11]
 [-2.46337804e+11]
 [ 3.31647955e+11]
 [ 9.72375293e+10]
 [ 2.65393714e+11]
 [-2.30913506e+12]
 [-6.04596139e+12]]
    TotalAmountofAssistance
0                 956

In [205]:
mse_norm = metrics.mean_squared_error(y_test, y_pred_norm)
print(math.sqrt(mse_norm))

1032310556577.1436


In [ ]:
#### Attempt adding CO2 emissions and number of investments
# sorted_X_drop_add = sorted_X_drop.copy(deep=True)


In [207]:
### Replace zeros with the means for each column
sorted_X_drop_filled = sorted_X_drop.mask(sorted_X_drop==0).fillna(sorted_X_drop.mean())

      BDPRP       CLPRB         ENPRP     GETCB     HYTCB     NCPRB  \
0     21.84   17747.000  23720.058432     186.0   14621.0   16298.0   
5   1943.76  331420.000  23720.058432     141.0   91902.0   92144.0   
10  4449.90    1819.000  23720.058432     808.0   33261.0   34194.0   
15    10.92  146450.000   3683.568000     345.0   60906.0  113693.0   
20  4078.62  310357.992  14842.800000  112878.0  128681.0  567365.0   

        NGMPB       NUETB      PAPRB     REPRB     SOTCB      TEPRB  \
0    380887.0  168290.108  1007501.0   23807.0       3.0  1429942.0   
5    182481.0  438728.000    55421.0  258955.0     101.0  1267004.0   
10  1031466.0  144716.000    35611.0  117840.0     124.0  1331452.0   
15       99.0  340153.000      212.0  128954.0   48229.0   615867.0   
20   271086.0  193529.000  1150741.0  737527.0  211839.0  2352883.0   

        TETCB     WDEXB     WDTCB    WSTCB     WWPRB       WYTCB  
0    614242.0  1395.604    6865.0    622.0    7488.0    1488.000  
5   1917138.

In [214]:
# print(sorted_X_drop_filled.head())
# print(sorted_X_drop.head())
# print((sorted_X_drop == 0).sum(axis=1))

0      3
5      3
10     3
15     1
20     2
      ..
229    7
234    5
239    4
244    4
249    5
Length: 250, dtype: int64


In [225]:
sorted_X_test_drop_filled = sorted_X_test_drop.mask(sorted_X_test_drop==0).fillna(sorted_X_test_drop.mean())

In [227]:
print((sorted_X_test_drop_filled == 0).sum(axis=1))

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
dtype: int64


In [218]:
linreg_drop_filled = linear_model.LinearRegression()
linreg_drop_filled.fit(sorted_X_drop_filled, y)

LinearRegression()

In [219]:
print(linreg_drop_filled.coef_)
print(linreg_drop_filled.intercept_)
print(linreg_drop_filled.score(sorted_X_drop_filled, y))

[[-6.78687076e+01  7.82218255e+00  1.77905493e+02  1.14757616e+02
   6.92068414e+01  7.16267985e+01  8.41314932e+00  9.94033165e+00
  -1.67220986e+00 -1.08839642e+02  1.81871744e+02 -7.41918723e+00
   8.59681088e+00  4.41167940e+03 -2.40256082e+02 -9.30243733e+02
   2.04892910e+02  7.71375869e+01]]
[-4736132.78732648]
0.19916387049455386


In [228]:
y_pred_drop_filled = linreg_drop_filled.predict(sorted_X_test_drop_filled)
print(y_pred_drop_filled)
print(y_test)

[[ 6.08547095e+06]
 [ 2.34385171e+07]
 [ 1.26487693e+07]
 [ 3.45003713e+07]
 [ 8.35617485e+07]
 [ 1.57661087e+07]
 [ 1.33969800e+07]
 [ 2.28906637e+07]
 [ 4.62142903e+07]
 [ 8.87184287e+07]
 [ 1.86526376e+07]
 [ 8.05300664e+06]
 [ 9.94765843e+06]
 [ 2.77031408e+07]
 [ 9.71598978e+06]
 [ 1.58900302e+07]
 [ 1.79890409e+07]
 [ 4.87691622e+07]
 [ 1.76506106e+07]
 [ 1.36049035e+07]
 [ 8.21789292e+06]
 [ 3.18727675e+06]
 [ 1.44566465e+07]
 [ 8.66302266e+06]
 [ 4.74650405e+07]
 [ 1.31935110e+07]
 [ 1.31882401e+08]
 [-9.10337233e+06]
 [ 1.05209597e+07]
 [ 1.28149608e+07]
 [ 2.65621077e+07]
 [ 1.61937350e+06]
 [ 2.72248349e+07]
 [ 1.31968685e+07]
 [ 1.77802500e+07]
 [ 1.68206158e+07]
 [ 1.15639630e+07]
 [ 5.96034130e+06]
 [ 1.72582018e+07]
 [ 5.10485462e+07]
 [ 1.07806048e+07]
 [ 1.31268647e+07]
 [ 7.15155979e+07]
 [ 1.85420176e+07]
 [ 3.67103881e+07]
 [ 1.57734832e+07]
 [ 2.81316478e+07]
 [ 7.29587111e+06]
 [ 1.85740444e+07]
 [ 5.64019825e+06]]
    TotalAmountofAssistance
0                 956

In [229]:
mse_drop_filled = metrics.mean_squared_error(y_test, y_pred_drop_filled)
print(math.sqrt(mse_drop_filled))

28641664.952540006
